### Integração Numérica - Regra de Simpson 3/8

Este notebook apresenta a implementação da Regra de Simpson 3/8 para cálculo de integrais definidas, incluindo o erro de aproximação simples e o erro generalizado, utilizando orientação a objetos.


In [19]:
%load_ext autoreload
%autoreload 2
from integral import Integral
import math
import sympy as sp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Classe `IntegralTresOitavosSimpson`

In [20]:
class IntegralTresOitavosSimpson(Integral):
    def __init__(self, x=None, y=None, limite_inf=None, limite_sup=None,
                 func_integral=None, h=None, erro_desejado=None):
        super().__init__()

        if erro_desejado is not None:
            if limite_inf is None or limite_sup is None or func_integral is None:
                raise ValueError("Para usar erro_desejado, forneça limite_inf, limite_sup e func_integral.")

            self.limite_inf = limite_inf
            self.limite_sup = limite_sup

            x_sym = sp.Symbol('x')
            func_lambd = sp.lambdify(x_sym, func_integral, modules=["math"])

            # Calcula K = max |f'''''(x)| em [a, b]
            K = self._derivada_quinta_simbolica(func_integral, limite_inf, limite_sup)

            if K is None:
                raise ValueError("Não foi possível calcular a quinta derivada simbolicamente.")

            # Fórmula erro Simpson 3/8: 
            # |Erro| <= (3 * (b - a) * h^5 / 80) * max |f'''''(x)|
            # Com h = (b - a) / n, rearranjando para n:
            n_quin = (( (3 * K * (limite_sup - limite_inf)**6) / (80 * erro_desejado) )) ** (1/5)
            n = math.ceil(n_quin)

            # Ajustar n para múltiplo de 3 (n % 3 == 0)
            while n % 3 != 0:
                n += 1

            n = min(max(n, 3), 10000)
            print(f"valor de n (subintervalos) encontrado: {n}")
            h = (limite_sup - limite_inf) / n
            print(f"valor de h calculado: {h}")

            self._setup(x, y, limite_inf, limite_sup, func_lambd, h)
        else:
            self._setup(x, y, limite_inf, limite_sup, func_integral, h)

    def _derivada_quinta_simbolica(self, func_expr, a, b):
        x = sp.Symbol('x')
        try:
            f_diff5 = sp.diff(func_expr, x, 5)
            f_diff5_func = sp.lambdify(x, f_diff5, modules=["math"])
            pontos = [a, (a + b) / 2, b]
            return max(abs(f_diff5_func(xi)) for xi in pontos)
        except Exception as e:
            print(f"[sympy fallback] Derivada quinta simbólica falhou: {e}")
            return None

    def integral(self):
        y_vals = self.y.copy()
        quant_x = len(self.x)

        if quant_x % 3 == 1:
            # Aplica regra 3/8 até penúltimos 3 pontos, e usa trapézio no fim
            b = self.x[-1]
            penultimo = self.x[-2]

            primeiro = y_vals[0]
            ultimo = y_vals[-1]
            y_interm = y_vals[1:-1]

            cte = (3 * self.h) / 8
            soma3x = 0
            soma2x = 0
            cont = 0
            for val in y_interm:
                if cont >= 2:
                    soma2x += 2 * val
                    cont = 0
                else:
                    soma3x += 3 * val
                    cont += 1

            return cte * (primeiro + soma2x + soma3x + ultimo)

        else:
            primeiro = y_vals[0]
            ultimo = y_vals[-1]
            y_interm = y_vals[1:-1]

            cte = (3 * self.h) / 8
            soma3x = 0
            soma2x = 0
            cont = 0
            for val in y_interm:
                if cont >= 2:
                    soma2x += 2 * val
                    cont = 0
                else:
                    soma3x += 3 * val
                    cont += 1

            return cte * (primeiro + soma3x + soma2x + ultimo)

    def erro_generalizado(self):
        d4_inf = self.derivada_quarta_ordem(self.limite_inf)
        d4_sup = self.derivada_quarta_ordem(self.limite_sup)
        maior = self.saber_maior(d4_inf, d4_sup)
        parcela_1 = (self.h**4 / 80) * (self.limite_sup - self.limite_inf)
        return abs(maior * parcela_1)

    def erro_simples(self):
        d4_inf = self.derivada_quarta_ordem(self.limite_inf)
        d4_sup = self.derivada_quarta_ordem(self.limite_sup)
        maior = self.saber_maior(d4_inf, d4_sup)
        parcela_1 = (3 * math.pow(self.h, 5)) / 80
        return abs(maior * parcela_1)

### Exemplo de uso com pontos

In [21]:
# Questão velocidade - lista

pontos_x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pontos_y = [0, 50.4, 53.7, 67.5, 74.3, 84.6, 92.1, 98.3, 100, 105, 110]

simpson_38 = IntegralTresOitavosSimpson(x=pontos_x, y=pontos_y)
print(f"Simpson 3/8 (x, y): {simpson_38.integral():.5f}")
print(f"Erro simples: {simpson_38.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_38.erro_generalizado():.5f}")

Simpson 3/8 (x, y): 758.66250
Erro simples: 329.76513
Erro generalizado: 1099.21709


In [22]:
# Questão margens do rio - lista

# Primeira parte da margem maior (M2)
pontos_x = [0, 10, 20, 30]
pontos_y = [113.6, 144.5, 185, 171.2]

simpson_38 = IntegralTresOitavosSimpson(x=pontos_x, y=pontos_y)
print(f"Simpson 3/8 parte 1 Margem 2 (x, y): {simpson_38.integral():.5f}")
print(f"Erro simples: {simpson_38.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_38.erro_generalizado():.5f}")
    # Soma do cálculo da parte 2 com Trapézio
margem2 = simpson_38.integral() + 2665.00000
print(f"Margem 2 resultante: {margem2:.5f}")

# Primeira parte da margem menor (M1)
pontos_x = [0, 10, 20, 30]
pontos_y = [50.8, 86.2, 136, 72.8]

simpson_38 = IntegralTresOitavosSimpson(x=pontos_x, y=pontos_y)
print(f"Simpson 3/8 parte 1 Margem 1(x, y): {simpson_38.integral():.5f}")
print(f"Erro simples: {simpson_38.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_38.erro_generalizado():.5f}")
    # Soma do cálculo da parte 2 com Trapézio
margem1 = simpson_38.integral() + 1238.00000
print(f"Margem 1 resultante: {margem1:.5f}")

# Resultado final
print(f"Resultado final: {(margem2 - margem1):.5f}")


Simpson 3/8 parte 1 Margem 2 (x, y): 4774.87500
Erro simples: 0.00000
Erro generalizado: 0.00000
Margem 2 resultante: 7439.87500
Simpson 3/8 parte 1 Margem 1(x, y): 2963.25000
Erro simples: 0.00000
Erro generalizado: 0.00000
Margem 1 resultante: 4201.25000
Resultado final: 3238.62500


### Exemplos de uso com função

In [23]:
# Questão 2: integral de 0 a 1 3*exp(-x) - Lista
def f(x):
    return 3 * math.exp(-x)

simpson_38_2 = IntegralTresOitavosSimpson(limite_inf=0, limite_sup=1, func_integral=f, h=1/6)
print(f"Simpson 3/8 (função): {simpson_38_2.integral():.5f}")
print(f"Erro simples: {simpson_38_2.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_38_2.erro_generalizado():.5f}")


Simpson 3/8 (função): 1.89638
Erro simples: 0.00001
Erro generalizado: 0.00003


In [24]:
x = sp.Symbol('x')
f_expr = sp.sqrt(x)

simpson_38 = IntegralTresOitavosSimpson(
    limite_inf=1,
    limite_sup=4,
    func_integral=f_expr,
    erro_desejado=0.01
)

print(f"Simpson 3/8 (x, y): {simpson_38.integral():.4f}")
print(f"Erro simples: {simpson_38.erro_simples():.4f}")
print(f"Erro generalizado: {simpson_38.erro_generalizado():.4f}")

valor de n (subintervalos) encontrado: 9
valor de h calculado: 0.3333333333333333
Simpson 3/8 (x, y): 4.6666
Erro simples: 0.0001
Erro generalizado: 0.0003
